In [1]:
import glob
import os

import autoeis as ae

from helpers import *


In [2]:
path_eis = "dataset/PJ131/eis-sorted/charge"
path_ecm = "dataset/PJ131/ecm/charge"

file_list = glob.glob(os.path.join(path_eis, "*.csv"))
file_list.sort(key=get_cycle_number)

fpath = file_list[0]
df = ae.io.load_eis_data(fpath, column_indices=[0, 1, 2])
freq = df["freq"].values
Zreal = df["Zreal"].values
Zimag = df["Zimag"].values
Z = Zreal + 1j * Zimag

ecms = ae.generate_equivalent_circuits(Z, freq, iters=50, tol=1e-4)


Circuit Evolution:   0%|          | 0/50 [00:00<?, ?it/s]

In [11]:
# Filter unphysical circuits
ohmic_resistance = ae.core.find_ohmic_resistance(Zreal, Zimag)
ecms_filtered = ae.core.apply_heuristic_rules(ecms.copy(), ohmic_resistance)
print(f"ECMs remaining after filtering: {len(ecms_filtered)}/{len(ecms)}")

# Save ECMs to disk, both raw and filtered
ecms.to_csv(os.path.join(path_ecm, os.path.basename(fpath)), index=False)
# ecms_filtered.to_csv(os.path.join(path_ecm, os.path.basename(fpath)))


ECMs remaining after filtering: 22/47
